In [15]:
import os
import sys
# reload(sys)
# sys.setdefaultencoding('utf-8')

import cPickle

import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
import gensim
from fuzzywuzzy import fuzz
from nltk.corpus import stopwords
from tqdm import tqdm
from scipy.stats import skew, kurtosis
from scipy.spatial.distance import cosine, cityblock, jaccard, canberra, euclidean, minkowski, braycurtis
from nltk import word_tokenize
stop_words = stopwords.words('english')

DATA_PATH = os.getcwd() + "/data"
FILE_NAME = 'features.csv'

In [2]:
train_df = pd.read_csv(os.path.join(DATA_PATH + '/train.csv'))
test_df = pd.read_csv(os.path.join(DATA_PATH + '/test.csv'))

train_sample_df = train_df.head()
test_sample_df = test_df.head()

In [3]:
train_sample_df

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [4]:
test_sample_df

,test_id,question1,question2
0,0,How does the Surface Pro himself 4 compare wit...,Why did Microsoft choose core m3 and not core ...
1,1,Should I have a hair transplant at age 24? How...,How much cost does hair transplant require?
2,2,What but is the best way to send money from Ch...,What you send money to China?
3,3,Which food not emulsifiers?,What foods fibre?
4,4,"How ""aberystwyth"" start reading?",How their can I start reading?


In [32]:
import spacy
nlp = spacy.load('en')

In [42]:
sent = u"There is an art, it says, or rather, a knack to flying." \
                 u"The knack lies in learning how to throw yourself at the ground and miss." \
                 u"In the beginning the Universe was created. This has made a lot of people "\
                 u"very angry and been widely regarded as a bad move."

parsed_review = nlp(sent)
print(parsed_review)

There is an art, it says, or rather, a knack to flying.The knack lies in learning how to throw yourself at the ground and miss.In the beginning the Universe was created. This has made a lot of people very angry and been widely regarded as a bad move.


In [43]:
for num, sentence in enumerate(parsed_review.sents):
    print 'Sentence {}:'.format(num + 1)
    print sentence
    print ''

Sentence 1:
There is an art, it says, or rather, a knack to flying.

Sentence 2:
The knack lies in learning how to throw yourself at the ground and miss.

Sentence 3:
In the beginning the Universe was created.

Sentence 4:
This has made a lot of people very angry and been widely regarded as a bad move.



In [44]:
for num, entity in enumerate(parsed_review.ents):
    print 'Entity {}:'.format(num + 1), entity, '-', entity.label_
    print ''

Entity 1: the beginning - DATE

Entity 2: Universe - ORG



In [54]:
token_text = [token.orth_ for token in parsed_review]
token_pos = [token.pos_ for token in parsed_review]

print set(token_pos)

pd.DataFrame(zip(token_text, token_pos),
             columns=['token_text', 'part_of_speech']).head()

set([u'ADV', u'NOUN', u'ADP', u'PUNCT', u'PROPN', u'DET', u'PART', u'PRON', u'VERB', u'CCONJ', u'ADJ'])


,token_text,part_of_speech
0,There,ADV
1,is,VERB
2,an,DET
3,art,NOUN
4,",",PUNCT


In [51]:
token_lemma = [token.lemma_ for token in parsed_review]
token_shape = [token.shape_ for token in parsed_review]

pd.DataFrame(zip(token_text, token_lemma, token_shape),
             columns=['token_text', 'token_lemma', 'token_shape']).head()

,token_text,token_lemma,token_shape
0,There,there,Xxxxx
1,is,be,xx
2,an,an,xx
3,art,art,xxx
4,",",",",","


In [50]:
token_entity_type = [token.ent_type_ for token in parsed_review]
token_entity_iob = [token.ent_iob_ for token in parsed_review]

pd.DataFrame(zip(token_text, token_entity_type, token_entity_iob),
             columns=['token_text', 'entity_type', 'inside_outside_begin']).head()

,token_text,entity_type,inside_outside_begin
0,There,,O
1,is,,O
2,an,,O
3,art,,O
4,",",,O


In [49]:
token_attributes = [(token.orth_,
                     token.prob,
                     token.is_stop,
                     token.is_punct,
                     token.is_space,
                     token.like_num,
                     token.is_oov)
                    for token in parsed_review]

df = pd.DataFrame(token_attributes,
                  columns=['text',
                           'log_probability',
                           'stop?',
                           'punctuation?',
                           'whitespace?',
                           'number?',
                           'out of vocab.?'])

df.loc[:, 'stop?':'out of vocab.?'] = (df.loc[:, 'stop?':'out of vocab.?']
                                       .applymap(lambda x: u'Yes' if x else u''))
                                               
df.head()

,text,log_probability,stop?,punctuation?,whitespace?,number?,out of vocab.?
0,There,-7.277903,Yes,,,,
1,is,-4.329765,Yes,,,,
2,an,-5.953294,Yes,,,,
3,art,-9.778431,,,,,
4,",",-3.391480,,Yes,,,


In [62]:
def pos_tagging(x):
    parsed = nlp(unicode(x, "utf-8"))
    return [token.pos_ for token in parsed]

test_sample_df["q1_pos"] = test_sample_df['question1'].apply(pos_tagging)
test_sample_df["q2_pos"] = test_sample_df['question2'].apply(pos_tagging)
test_sample_df

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,test_id,question1,question2,q1_pos,q2_pos
0,0,How does the Surface Pro himself 4 compare wit...,Why did Microsoft choose core m3 and not core ...,"[ADV, VERB, DET, PROPN, PROPN, PRON, NUM, VERB...","[ADV, VERB, PROPN, VERB, ADJ, NOUN, CCONJ, ADV..."
1,1,Should I have a hair transplant at age 24? How...,How much cost does hair transplant require?,"[VERB, PRON, VERB, DET, NOUN, NOUN, ADP, NOUN,...","[ADV, ADJ, NOUN, VERB, NOUN, NOUN, NOUN, PUNCT]"
2,2,What but is the best way to send money from Ch...,What you send money to China?,"[NOUN, CCONJ, VERB, DET, ADJ, NOUN, PART, VERB...","[NOUN, PRON, VERB, NOUN, ADP, PROPN, PUNCT]"
3,3,Which food not emulsifiers?,What foods fibre?,"[ADJ, NOUN, ADV, NOUN, PUNCT]","[NOUN, NOUN, NOUN, PUNCT]"
4,4,"How ""aberystwyth"" start reading?",How their can I start reading?,"[ADV, PUNCT, ADJ, PUNCT, NOUN, VERB, PUNCT]","[ADV, ADJ, VERB, PRON, VERB, VERB, PUNCT]"


In [65]:
import gensim
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [ ]:
model = gensim.models.KeyedVectors.load_word2vec_format('data/GoogleNews-vectors-negative300.bin.gz', binary=True)
data['wmd'] = data.apply(lambda x: wmd(x['question1'], x['question2']), axis=1)